In [89]:
from graphdatascience import GraphDataScience
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import altair as alt

graph_name = "testgraph" # project graph name

gds = GraphDataScience("bolt://localhost:7687", auth=("neo4j", "j4oenj4oen"))
gds.version()

'2.5.4'

In [90]:
query = """
CALL apoc.cypher.runFile("https://raw.githubusercontent.com/smallcat9603/graph/main/cypher/euro-roads.cypher")
"""
gds.run_cypher(query)

,row,result
0,-1,"{'constraintsRemoved': 0, 'indexesRemoved': 0,..."


In [91]:
node_projection = ["Place"]
# # why raising error "java.lang.UnsupportedOperationException: Loading of values of type StringArray is currently not supported" ???
# node_projection = {"Query": {"properties": 'phrase'}, "Article": {"properties": 'phrase'}, "Noun": {}}
relationship_projection = {
    "EROAD": {"orientation": "UNDIRECTED"},
    # "CORRELATES": {"orientation": "UNDIRECTED", "properties": ["common"]} # Unsupported type [TEXT_ARRAY] of value StringArray[DNP]. Please use a numeric property.
    }
# # how to project node properties???
# node_properties = { 
#     "nodeProperties": {
#         "phrase": {"defaultValue": []},
#         "salience": {"defaultValue": []}
#     }
# }

exists_result = gds.graph.exists(graph_name)
if exists_result["exists"]:
    G = gds.graph.get(graph_name)
    G.drop()
G, result = gds.graph.project(graph_name, node_projection, relationship_projection)

In [92]:
result_fastRP_stream = gds.fastRP.stream(
    G,
    randomSeed=42,
    embeddingDimension=10,
)
for i in range(5):
    print(gds.util.asNode(result_fastRP_stream['nodeId'][i])["name"])
    print(result_fastRP_stream['embedding'][i])

gds.fastRP.write(
    G,
    randomSeed=42,
    embeddingDimension=256,
    writeProperty="embedding"
)


Larne
[0.414510577917099, -0.1798122525215149, 0.6471794843673706, -0.29909804463386536, 1.1295080184936523, 0.11190766096115112, -0.051343806087970734, -0.18687719106674194, 0.32004866003990173, -0.34158825874328613]
Belfast
[0.4128932058811188, -0.19002306461334229, 0.8135403990745544, -0.3874449133872986, 1.1655800342559814, 0.11184898763895035, 0.10684403032064438, -0.17019778490066528, 0.46915024518966675, -0.17866307497024536]
Dublin
[0.16903702914714813, -0.0938960388302803, 0.9079959392547607, -0.7445522546768188, 0.4562141001224518, -0.13763055205345154, 0.40263932943344116, -0.21697774529457092, 0.5429133176803589, 0.618682861328125]
Wexford
[-0.21123552322387695, -0.46115362644195557, 0.6048547625541687, -0.2308708131313324, 0.012157291173934937, -0.15247617661952972, 0.11909198015928268, -0.3633899390697479, 0.4350880980491638, 1.1503161191940308]
Rosslare
[-0.47401195764541626, -0.6213488578796387, 0.4072519540786743, -0.10329093039035797, -0.17384669184684753, -0.10557685

nodeCount                                                              894
nodePropertiesWritten                                                  894
preProcessingMillis                                                      0
computeMillis                                                            6
writeMillis                                                             30
configuration            {'writeConcurrency': 4, 'nodeSelfInfluence': 0...
Name: 0, dtype: object

In [93]:
query = """
MATCH (p:Place)-[:IN_COUNTRY]->(country)
WHERE country.code IN ["E", "GB", "F", "TR", "I", "D", "GR"]
RETURN p.name AS place, p.embedding AS embedding, country.code AS country
"""
result = gds.run_cypher(query)
result.head()

,place,embedding,country
0,Folkestone,"[-0.021473096683621407, -0.1141786128282547, -...",GB
1,Holyhead,"[0.12448534369468689, -0.05214271321892738, -0...",GB
2,London,"[0.03214350342750549, -0.06454835832118988, -0...",GB
3,Liverpool,"[-0.045100919902324677, 0.022782577201724052, ...",GB
4,Colchester,"[0.02683684043586254, -0.10858513414859772, -0...",GB


In [94]:
X_embedded = TSNE(n_components=2, random_state=6).fit_transform(np.array(list(result["embedding"])))

places = result["place"]
tsne_df = pd.DataFrame(data = {
    "place": places,
    "country": result["country"],
    "x": [value[0] for value in X_embedded],
    "y": [value[1] for value in X_embedded]
})
tsne_df.head()

,place,country,x,y
0,Folkestone,GB,-5.157909,-11.080489
1,Holyhead,GB,25.528675,23.263361
2,London,GB,-5.962321,-9.515672
3,Liverpool,GB,21.729271,22.101677
4,Colchester,GB,-7.654853,-11.298371


In [95]:
alt.Chart(tsne_df).mark_circle(size=60).encode(
    x='x',
    y='y',
    color='country',
    tooltip=['place', 'country']
).properties(width=700, height=400)

alt.Chart(...)

In [96]:
exists_result = gds.graph.exists(graph_name)
if exists_result["exists"]:
    G = gds.graph.get(graph_name)
    G.drop()    
query = """
MATCH (n) DETACH DELETE n
"""
gds.run_cypher(query)

""
